In [1]:
# !pip install pyswarm
# !pip install --upgrade pandas
# !pip install --upgrade pandas-datareader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import pandas_datareader as pdr
from __future__ import division
import random
import math
import time
from pyswarm import pso
from sklearn.svm import SVR
from sklearn.model_selection import TimeSeriesSplit

In [2]:
comlist = ['ADOBE','AMERICAN','APPLE','AT&T','BANK OF','COCA-COL','EXXON MO','FMC','HEWLETT','HONEYWELL','ORACLE','TESLA']
sharelist = ['ADBE','AXP','AAPL','T','BK','KO','XOM','FMC','HPE','HON','ORCL','TSLA']
for comlistitr in range(len(comlist)):
  com = comlist[comlistitr]
  share = sharelist[comlistitr]
  start = '2016-01-04'
  end = '2020-10-12'
  data = pdr.get_data_yahoo(share, start=start, end=end)
  print('company = ',com)
  for comresultitr in range(5):
    print('iteration ',comresultitr+1)
    # MF
    def MF(high, low, close, volume):
        mf = (high + low + close) / 3 * volume
        return mf 
    # EMA
    def EMA(data):
        data_float = data.astype(float)
        return data_float.ewm(span = 20).mean()
    # MFI
    def MFI(data):
        returns = data - data.shift(1)
        returns.dropna()
        up, down = returns.copy(), returns.copy()
        up[up<0] = 0
        down[down>0] = 0
        upm = up.mean()
        downm = down.abs().mean()
        
        try:
          mfi = 100 - 100 / (1 + upm / downm)
          return mfi
        except ZeroDivisionError:
          return 0
    # RSI
    def RSI(data):
        returns = data - data.shift(1)
        returns.dropna()
        up,down = returns.copy(), returns.copy()
        up[up<0] = 0
        down[down>0] = 0
        upm = up.mean()
        downm = down.abs().mean()
        try:
          rsi = 100 - 100 / (1 + upm / downm)
          return rsi
        except ZeroDivisionError:
          return 0 
    # MACD
    def MACD(ema):
        high = 0.075 * ema
        low = 0.15 * ema
        macd = high - low
        return macd
    def oscillator(data):
        curr = data[-1]
        data = list(data)
        high = max(data)
        low = min(data)
        so = (curr - low) / (high - low) * 100 
        return so
    data['MF'] = MF(data['High'], data['Low'], data['Close'], data['Volume'])
    data['EMA'] = EMA(data.iloc[:, [3]])
    data['RSI'] = data['Close'].rolling(7).apply(RSI)
    data['MFI'] = data['MF'].rolling(7).apply(MFI)
    data['SO'] = data['Close'].rolling(7).apply(oscillator)
    data['MACD'] = MACD(data.iloc[:,[7]])
    data['MACD_Signal'] = MACD(data.iloc[:, [11]])

    data.drop(['Adj Close', 'MF', 'MACD_Signal'], axis=1, inplace=True)
    xdata = data.dropna()
    ydata = pd.DataFrame(xdata['Close'].shift(-7))
    ydata.dropna(inplace=True)
    xdata  = xdata[:-7]
    xcols = xdata.columns
    xdates = xdata.index

    ycols = ydata.columns
    ydates = ydata.index

    from sklearn.preprocessing import StandardScaler
    xscaler = StandardScaler()
    yscaler = StandardScaler()
    xdata = pd.DataFrame(xscaler.fit_transform(xdata))
    ydata = pd.DataFrame(yscaler.fit_transform(ydata))

    xdata.columns = xcols
    xdata.index = xdates

    ydata.columns = ycols
    ydata.index = ydates
    x_trains, y_trains = xdata[:900], ydata[:900]
    y_trains = pd.DataFrame(y_trains)
    x_tests, y_tests = xdata[900:], ydata[900:]
    y_tests = pd.DataFrame(y_tests)
    y_tests.dropna(inplace=True)

    """Least Squares Support Vector Regression."""
    import numpy as np
    from sklearn.base import BaseEstimator, RegressorMixin
    from sklearn.metrics.pairwise import rbf_kernel
    from sklearn.gaussian_process import kernels
    from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
    from scipy.sparse import linalg


    class LSSVR1(BaseEstimator, RegressorMixin):
        def __init__(self, C=None, kernel=None, gamma=None):
            self.supportVectors      = None
            self.supportVectorLabels = None
            self.C = C
            self.gamma = gamma
            self.kernel= kernel
            self.idxs  = None
            self.K = None
            self.bias = None 
            self.alphas = None
                

        def set_params(self, **parameters):
            for parameter, value in parameters.items():
                setattr(self, parameter, value)
            return self

        def fit(self, x_train, y_train):
            
            if type(self.idxs) == type(None):
                self.idxs=np.full(x_train.shape[0],True, dtype=bool)
                
                
            self.supportVectors      = x_train.loc[self.idxs, :]
            self.supportVectorLabels = y_train.loc[self.idxs]
                  
            K = self.kernel_func(self.kernel, x_train, self.supportVectors, self.gamma)
            
            self.K = K
            OMEGA = K
            OMEGA[self.idxs, np.arange(OMEGA.shape[1])] =OMEGA[self.idxs, np.arange(OMEGA.shape[1])] + 1/self.C
            
            D = np.zeros(np.array(OMEGA.shape) + 1)
            
            D[1:,1:] = OMEGA
            D[0, 1:] += 1
            D[1:,0 ] += 1

            n = len(self.supportVectorLabels) + 1
            t = np.zeros(n)
            t = pd.DataFrame(t)
            t[1:n] = self.supportVectorLabels.values
        
            try:
                z = linalg.lsmr(D.T, t)[0]
            except:
                z = np.linalg.pinv(D).T @ t.ravel()

            self.bias   = z[0]
            self.alphas = z[1:]
            self.alphas = self.alphas[self.idxs]

            return self

        def predict(self, x_test):
            K = self.kernel_func(self.kernel, x_test, self.supportVectors, self.gamma)

            return (K @ self.alphas) + self.bias

        def kernel_func(self, kernel, u, v, gamma):
            if kernel == 'linear':
                k = np.dot(u, v.T)
            if kernel == 'rbf':
                k = rbf_kernel(u, v, gamma=gamma)
            return k

    class sca:
      def optimize(self,obj,dim,bound,n,max_it):
        #set variable values
        num_agents = n
        num_features = 2
        max_iter = max_it

        #initialize
        population = np.random.uniform(low=0,high=1,size=(num_agents,num_features))
        fitness = np.zeros(num_agents)
        gbest = np.zeros(num_features)
        gbest_fitness = np.inf

        for i in range(num_agents):
          fitness[i] = obj(population[i])
        sort_order = np.argsort(fitness)
        fitness = fitness[sort_order]
        population = population[sort_order]
        gbest = population[0].copy()
        gbest_fitness = fitness[0]

        #parameters
        a = 3

        for itr in range(max_iter):
          r1 = a-itr*(a/max_iter)
          for i in range(num_agents):
            for j in range(num_features):
              r2 = (2 * np.pi) * np.random.random()
              r3 = 2 * np.random.random()
              r4 = np.random.random()
              if r4 < 0.5:
                population[i][j] += (r1*np.sin(r2)*abs(r3*gbest[j]-population[i][j]))
              else:
                population[i][j] += (r1*np.cos(r2)*abs(r3*gbest[j]-population[i][j]))
          for i in range(num_agents):
            fitness[i] = obj(population[i])
          
          #sort
          sort_order = np.argsort(fitness)
          fitness = fitness[sort_order]
          population = population[sort_order]

          #update gbest
          if fitness[0] < gbest_fitness:
            gbest = population[0].copy()
            gbest_fitness = fitness[0]
        for i in range(num_features):
          if gbest[i] > 1:
            gbest[i] = 1
          elif gbest[i] < 0:
            gbest[i] = 0
        return gbest,gbest_fitness

    fold_count=10 

    def load_csvdata():
        global X
        global Y
        
        X = xdata
        Y = ydata

    def svrPso(params,problem_size = 2):
        kf = TimeSeriesSplit(n_splits=fold_count)
        for train, test in kf.split(X):
            Total = 0
            
            X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], Y.iloc[train], Y.iloc[test]
            nn = LSSVR1(kernel = 'rbf',C=params[0], gamma = params[1])
            nn.fit(X_train,y_train)
            result = nn.predict(X_test);
            thisError = calsError(y_test, result)
            Total = Total + thisError    
        ErrorCV = Total/fold_count; 
        # print('Optimizing the Parameters ..... C = {c}, gamma={e}, MSE={m}'.format(c=params[0], e=params[1], m=ErrorCV))
        return ErrorCV

    def calsError(y_test, result):
        y_pred = pd.DataFrame(result)
        y_pred.index = y_test.index
        data = pd.concat([y_test,y_pred],axis=1)
        data['Error'] = (data['Close'] - data[0])**2
        err = np.sqrt(np.sum(data['Error']))
        
        return err
        
    def main_run(algo):
        
            load_csvdata()
            if  algo == sca:
                bounds=[(0.01,1000),(0.0001,500)] #hybridized
                opt=sca()
                num_sca=20
                num_iter=1
                loc,fit = opt.optimize(svrPso,2,bounds, num_sca,num_iter)
                loc[0] = loc[0] * 1000 #scale to [0,1000]
                if loc[0] < 0.01:
                  loc[0] = 0.01
                loc[1] *= 500
                if loc[1] < 0.0001:
                  loc[1] = 0.0001
            return loc,fit

    algo = sca
    loc,fit = main_run(algo)

    print("{:10.8} {:10.6} {:10.10} {:10.10} {:10.10}".format(com,share,loc[0],loc[1],fit))

company =  ADOBE
iteration  1
ADOBE      ADBE       492.5967918 1.514572658 0.4878112734
iteration  2


IndexError: positional indexers are out-of-bounds